# 個体間関係のラベル付け

## おまじないパート

In [1]:
import sys
from pathlib import Path

# 現在のノートブックのパスを取得
notebook_dir = Path().resolve() 

# プロジェクトルートディレクトリ（notebooksディレクトリの親ディレクトリ）を取得
# プロジェクトルートは3階層上です
project_root = notebook_dir.parent.parent.parent

# プロジェクトルートをPythonの検索パスに追加
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [2]:
from groom import GroomAim2, \
    save_to_parquet, load_from_parquet, \
        save_model, load_model

In [3]:
self = GroomAim2()

## data準備

### raw_nose_face_dfの読み込み

In [4]:
raw_nose_face_df = load_from_parquet(
    "../../../data/03_aim_1_data/corrected_nose_face_df.parquet"
)

# 不要な列を削除
raw_nose_face_df = raw_nose_face_df.drop(
    columns=['t0_flag', 
             'delta_face','delta_nose',
             'shade', 'shade_condition', 
             'is_role_swapped', 'has_Sg',
             'nose-face'], 
    errors='ignore'
)

--- 復元完了 ---
復元されたDataFrameのshape: (80367, 21)


In [5]:
print(raw_nose_face_df.columns)

Index(['delta_time', 'sampling_id', 'datetime', 'name', 'id', 'sex',
       'behavior', 'to', 'from', 'kin', 'corrected_delta_nose',
       'corrected_delta_face', 'corrected_nose-face'],
      dtype='object')


In [6]:
# 'from'列と'to'列の両方に値がある（NaNでない）行を抽出
filtered_df = raw_nose_face_df.dropna(subset=['from', 'to'])

# 列を指定して表示
print(filtered_df[['sampling_id', 'name', 'from', 'to']])

Empty DataFrame
Columns: [sampling_id, name, from, to]
Index: []


### grooming_dfとgroomed_dfの作成

#### まずは分割

In [7]:
raw_grooming_df = raw_nose_face_df[raw_nose_face_df['behavior']=='grooming']
raw_groomed_df = raw_nose_face_df[raw_nose_face_df['behavior']=='groomed']

### fromおよびto列の補完

In [8]:
grooming_df = self.fix_grooming_records(
    raw_grooming_df,
    'grooming'
)

--- Mode: grooming (Counterpart column: to) ---
記録ミスの可能性がある sampling_id: [8, 73]
入れ替え処理が完了しました。
from 列を name 列の値で補完しました。

[処理結果の確認(代表各1行)]
      sampling_id      name      from        to
602             3     Botan     Botan   Komatsu
1204            5     Botan     Botan   Komatsu
1806            8     Botan     Botan   Komatsu
2408           10     Botan     Botan  Botaninf
3311           15     Kanna     Kanna      Yuna
4214           20  Tsutsuji  Tsutsuji      Gure
4515           22  Tsutsuji  Tsutsuji      Kobu
4816           24  Tsutsuji  Tsutsuji      Kobu
5117           26  Tsutsuji  Tsutsuji     Tsuwa
5719           28  Tsutsuji  Tsutsuji     Tsuwa


In [9]:
groomed_df = self.fix_grooming_records(
    raw_groomed_df,
    'groomed'
)

--- Mode: groomed (Counterpart column: from) ---
記録ミスの可能性がある sampling_id: [7, 51, 72]
入れ替え処理が完了しました。
to 列を name 列の値で補完しました。

[処理結果の確認(代表各1行)]
       sampling_id      name     from        to
903              4     Botan  Komatsu     Botan
1505             7     Botan  Komatsu     Botan
3612            17     Kanna     Kobu     Kanna
5418            27  Tsutsuji    Tsuwa  Tsutsuji
7224            33     Tsuwa  Tsukasa     Tsuwa
7826            35     Tsuwa  Tsukasa     Tsuwa
9632            42    Takana     Tabu    Takana
10234           44    Binega     Hiba    Binega
11739           51   Komatsu   Namiru   Komatsu
12642           56      Yuna    Kanna      Yuna


### ペアの入力

### 個体間関係の結果の読み込み